<a href="https://colab.research.google.com/github/sakibchowdhury131/railcop/blob/main/timeSeriesDL_TEST1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow==2.2.0

     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 3.0 MB 30.9 MB/s 
     |████████████████████████████████| 454 kB 70.3 MB/s 
     |████████████████████████████████| 2.9 MB 42.4 MB/s 


In [ ]:
import numpy as np
np.random.seed(1)

import tensorflow as tf
tf.random.set_seed(1)

In [ ]:
import os
from tensorflow import keras
import matplotlib.pyplot as plt
import math
import pandas as pd

In [ ]:
label0 = pd.read_csv('/content/drive/MyDrive/RailCop/New Files/Sakib/data/label0/Copy of train_1.csv')
label1 = pd.read_csv('/content/drive/MyDrive/RailCop/New Files/Sakib/data/label1/Copy of train_1.csv')
print(label1)

      Sensor1  Sensor2  ...  Sensor2+Sensor4  Sensor1+Sensor2+Sensor3
0           2        9  ...               10                       12
1           4        9  ...                9                       13
2           4        9  ...               11                       15
3           4       11  ...               12                       16
4           2        9  ...               10                       12
...       ...      ...  ...              ...                      ...
5384        3        9  ...               10                       13
5385        3       10  ...               11                       14
5386        4        8  ...                9                       13
5387        3        9  ...               10                       13
5388        3       10  ...               12                       15

[5389 rows x 7 columns]


In [ ]:
__fullVector0 = label0['Sensor1+Sensor2+Sensor3'].to_numpy()
__fullVector1 = label1['Sensor1+Sensor2+Sensor3'].to_numpy()


print(__fullVector0.shape)
print(__fullVector1.shape)


(23050,)
(5389,)


In [ ]:
__frameSize = 50

## label 0 framing

__numFrames0 = int(__fullVector0.shape[0] / __frameSize)
__frames0 = np.zeros((__numFrames0,__frameSize))
for i in range (0, __numFrames0):
  for j in range (0, __frameSize):
    __frames0[i][j] = __fullVector0[i*__frameSize+j]

In [ ]:
## label 1 framing

__numFrames1 = int(__fullVector1.shape[0] / __frameSize)
__frames1 = np.zeros((__numFrames1,__frameSize))
for i in range (0, __numFrames1):
  for j in range (0, __frameSize):
    __frames1[i][j] = __fullVector1[i*__frameSize+j]

In [ ]:
__frames0.shape

(461, 50)

In [ ]:
__frames1.shape

(107, 50)

In [ ]:
__maxFrames = __frames1.shape[0]
__label0 = __frames0[0:__maxFrames, :]
__label1 = __frames1[0:__maxFrames, :]

In [ ]:
Y0 = np.zeros(__label0.shape[0])
Y1 = np.ones(__label1.shape[0])
print(Y0.shape)
print(Y1.shape)

(107,)
(107,)


In [ ]:
X = np.asarray(np.concatenate((__label0, __label1), axis = 0), dtype=np.float32)
Y = np.asarray(np.concatenate((Y0, Y1), axis = 0), dtype = np.float32)
print(X.shape)
print(Y.shape)

(214, 50)
(214,)


In [ ]:
from sklearn.utils import shuffle
X, Y = shuffle(X,Y)

In [ ]:
def network():
  __input = keras.Input((X.shape[1], 1))
  __conv1 = keras.layers.Conv1D(filters = 8, kernel_size = 5, activation = 'relu')(__input)
  __conv2 = keras.layers.Conv1D(filters = 8, kernel_size = 5, activation = 'relu')(__conv1)
  __dense3 = keras.layers.Dense(16, activation = 'relu')(__conv2)
  __output = keras.layers.Dense(1, activation = 'sigmoid')(__dense3)

  model = keras.Model(inputs = __input, outputs = __output)
  return model 



model = network()
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 50, 1)]           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 46, 8)             48        
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 42, 8)             328       
_________________________________________________________________
dense_10 (Dense)             (None, 42, 16)            144       
_________________________________________________________________
dense_11 (Dense)             (None, 42, 1)             17        
Total params: 537
Trainable params: 537
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
                      optimizer= keras.optimizers.Adam(lr=0.0008),
                      metrics=['accuracy'])

In [ ]:
__modelVersion = '__timeSeriesTest1.0'

weight_saver = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/RailCop/New Files/Sakib/models/'+__modelVersion, monitor='val_accuracy', 
                                save_best_only=True, save_weights_only=False, mode= 'max')

annealer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.00001,mode= 'min')


hist = model.fit( x = X , 
                  y = Y, 
                  batch_size = 64,
                  epochs = 600,
                  callbacks = [weight_saver , annealer],
                  validation_split = 0.33,
                  shuffle = True,
                  verbose = 1)

Epoch 1/600
3/3 [==============================] - 1s 341ms/step - loss: 0.6965 - accuracy: 0.4950 - val_loss: 0.6994 - val_accuracy: 0.4842 - lr: 8.0000e-04
Epoch 2/600
3/3 [==============================] - 0s 16ms/step - loss: 0.6937 - accuracy: 0.5087 - val_loss: 0.6998 - val_accuracy: 0.4742 - lr: 8.0000e-04
Epoch 3/600
3/3 [==============================] - 0s 16ms/step - loss: 0.6935 - accuracy: 0.5183 - val_loss: 0.7051 - val_accuracy: 0.4594 - lr: 8.0000e-04
Epoch 4/600
3/3 [==============================] - 0s 14ms/step - loss: 0.6929 - accuracy: 0.5246 - val_loss: 0.7084 - val_accuracy: 0.4581 - lr: 8.0000e-04
Epoch 5/600
3/3 [==============================] - 0s 16ms/step - loss: 0.6945 - accuracy: 0.5225 - val_loss: 0.7124 - val_accuracy: 0.4527 - lr: 8.0000e-04
Epoch 6/600
3/3 [==============================] - 0s 16ms/step - loss: 0.6943 - accuracy: 0.5228 - val_loss: 0.7095 - val_accuracy: 0.4554 - lr: 8.0000e-04
Epoch 7/600
3/3 [==============================] - 0s 17m